# اپلیکیشن آنلاین استعداد‌یابی رشته فوتبال  — پروژه نهایی درس یادگیری ماشین

**دانشگاه آزاد اسلامی – واحد علوم و تحقیقات**  
**درس:** یادگیری ماشین  
**استاد:** دکتر مهدی اسلامی

**نام و نام خانوادگی:** علی اللهی قلی

**شماره دانشجویی:** 40112340119097

**تاریخ:** 1404-10-06

## چرا این نسخه؟
در Colab جدید (Python 3.12) پکیج `mediapipe` گاهی namespace قدیمی `mp.solutions` را ندارد و باعث خطا می‌شود.  
برای اینکه پروژه **حتماً اجرا شود**، در این نسخه از **MoveNet (TensorFlow Lite)** برای Pose Estimation استفاده می‌کنیم.

✅ سبک و سریع (Lightning)  
✅ قابل اجرا روی CPU  
✅ مناسب برای پروژه دانشگاهی و “مدل کوچک و قابل حمل”


## 1) نصب کتابخانه‌ها

In [1]:
!pip -q install gradio opencv-python-headless pandas numpy scikit-learn
# TensorFlow معمولاً روی Colab از قبل نصب است

## 2) ایمپورت‌ها + دانلود مدل MoveNet (TFLite)

In [2]:

import gradio as gr
import pandas as pd
import numpy as np
import cv2
import tensorflow as tf

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, Ridge

print("TensorFlow:", tf.__version__)

# دانلود مدل MoveNet Lightning (TFLite - float16)
# منبع رسمی TFHub (همان لینک داخل کولب رسمی TensorFlow)
!wget -q -O model.tflite "https://tfhub.dev/google/lite-model/movenet/singlepose/lightning/tflite/float16/4?lite-format=tflite"

interpreter = tf.lite.Interpreter(model_path="model.tflite")
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

INPUT_SIZE = input_details[0]['shape'][1]  # 192
print("MoveNet input size:", INPUT_SIZE)


TensorFlow: 2.19.0
MoveNet input size: 192


/usr/local/lib/python3.12/dist-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


## 3) مرحله ۱: پیشنهاد پست از داده متنی

In [3]:

def recommend_position_text(age, height_cm, weight_kg, preferred_foot,
                            sprint_30m_sec, vertical_jump_cm,
                            coopers_m, agility_t_sec):

    bmi = weight_kg / ((height_cm/100)**2 + 1e-9)
    score = {"GK":0, "DEF":0, "MID":0, "FWD":0}

    # GK
    if height_cm >= 185: score["GK"] += 2
    if vertical_jump_cm >= 45: score["GK"] += 2
    if agility_t_sec <= 11.5: score["GK"] += 1

    # FWD
    if sprint_30m_sec <= 4.3: score["FWD"] += 3
    if vertical_jump_cm >= 50: score["FWD"] += 2

    # MID
    if coopers_m >= 2700: score["MID"] += 3
    if agility_t_sec <= 10.8: score["MID"] += 2

    # DEF
    if height_cm >= 178: score["DEF"] += 2
    if 20 <= bmi <= 27: score["DEF"] += 1
    if coopers_m >= 2400: score["DEF"] += 1

    best = max(score, key=score.get)
    fa = {"GK":"دروازه‌بان","DEF":"مدافع","MID":"هافبک","FWD":"مهاجم"}

    explanation = f"""امتیازها: {score}
BMI: {bmi:.1f}
پای غالب: {preferred_foot}
پیشنهاد مرحله ۱: {fa[best]}
"""
    return fa[best], explanation


## 4) مرحله ۲: Pose Estimation با MoveNet (عکس/ویدئو)

In [4]:

# ترتیب کلیدپوینت‌ها در MoveNet (17 نقطه)
# 0:nose 1:left_eye 2:right_eye 3:left_ear 4:right_ear
# 5:left_shoulder 6:right_shoulder 7:left_elbow 8:right_elbow 9:left_wrist 10:right_wrist
# 11:left_hip 12:right_hip 13:left_knee 14:right_knee 15:left_ankle 16:right_ankle

LEFT_HIP = 11
RIGHT_HIP = 12

def _movenet_infer(bgr_img):
    # پیش‌پردازش: resize + تبدیل به int32 (0..255)
    img = cv2.cvtColor(bgr_img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (INPUT_SIZE, INPUT_SIZE))
    input_image = np.expand_dims(img, axis=0).astype(np.int32)

    interpreter.set_tensor(input_details[0]['index'], input_image)
    interpreter.invoke()
    keypoints = interpreter.get_tensor(output_details[0]['index'])  # [1,1,17,3] => y,x,score
    return keypoints[0,0,:,:]  # [17,3]

def analyze_pose_image(image_path):
    image = cv2.imread(image_path)
    if image is None:
        return None, "تصویر خوانده نشد (فرمت/مسیر مشکل دارد)."

    kps = _movenet_infer(image)
    # hip_sway = |x_left_hip - x_right_hip|
    hip_sway = float(abs(kps[LEFT_HIP,1] - kps[RIGHT_HIP,1]))
    conf_l = float(kps[LEFT_HIP,2])
    conf_r = float(kps[RIGHT_HIP,2])

    feats = {"hip_sway_mean": hip_sway, "hip_conf": float((conf_l+conf_r)/2)}
    summary = f"Pose از تصویر استخراج شد ✅ | hip_sway_mean={hip_sway:.4f} | hip_conf={feats['hip_conf']:.2f}"
    return feats, summary

def analyze_pose_video(video_path, max_seconds=12, sample_every=3):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        return None, "ویدئو باز نشد (فرمت/مسیر مشکل دارد)."

    fps = cap.get(cv2.CAP_PROP_FPS) or 30
    max_frames = int(max_seconds * fps)

    hip_vals, conf_vals = [], []
    frame_idx = 0

    while True:
        ok, frame = cap.read()
        if not ok:
            break
        frame_idx += 1
        if frame_idx > max_frames:
            break
        if frame_idx % sample_every != 0:
            continue

        kps = _movenet_infer(frame)
        hip = float(abs(kps[LEFT_HIP,1] - kps[RIGHT_HIP,1]))
        conf = float((kps[LEFT_HIP,2] + kps[RIGHT_HIP,2]) / 2)
        hip_vals.append(hip)
        conf_vals.append(conf)

    cap.release()

    if len(hip_vals) < 3:
        return None, "Pose کافی استخراج نشد (۸–۱۲ ثانیه، تمام‌قد، نور خوب)."

    feats = {"hip_sway_mean": float(np.mean(hip_vals)), "hip_conf": float(np.mean(conf_vals))}
    summary = f"Pose از ویدئو استخراج شد ✅ | hip_sway_mean={feats['hip_sway_mean']:.4f} | hip_conf={feats['hip_conf']:.2f}"
    return feats, summary


## 5) مرحله ۳: آموزش مدل‌ها (حالت ۱ و ۳) از CSV

In [5]:

def train_models_from_csv(csv_file_obj):
    csv_path = csv_file_obj.name if hasattr(csv_file_obj, "name") else str(csv_file_obj)
    df = pd.read_csv(csv_path)

    features = ["age","height_cm","weight_kg","sprint_30m_sec","vertical_jump_cm","agility_t_sec","coopers_m"]
    required = features + ["academy_accept","speed_score","power_score","endurance_score"]
    for c in required:
        if c not in df.columns:
            raise ValueError(f"ستون {c} در CSV وجود ندارد.")

    X = df[features].astype(float)

    clf = Pipeline([("scaler", StandardScaler()), ("lr", LogisticRegression(max_iter=400))])
    clf.fit(X, df["academy_accept"].astype(int))

    reg = Pipeline([("scaler", StandardScaler()), ("ridge", Ridge(alpha=1.0))])
    reg.fit(X, df[["speed_score","power_score","endurance_score"]].astype(float))

    return clf, reg


## 6) اجرای اپ Gradio (مرحله ۱ + ۲ + ۳)

In [6]:

clf_model, reg_model = None, None
last_pose_feats = None

def ui_pose(img_path, vid_path):
    global last_pose_feats
    if img_path is not None:
        feats, summary = analyze_pose_image(img_path)
    elif vid_path is not None:
        feats, summary = analyze_pose_video(vid_path)
    else:
        feats, summary = None, "هیچ فایل تصویری/ویدئویی آپلود نشده است."
    last_pose_feats = feats
    return summary

def ui_train(csv_file):
    global clf_model, reg_model
    clf_model, reg_model = train_models_from_csv(csv_file)
    return "✅ مدل‌های حالت ۱ و ۳ آموزش داده شدند."

def ui_predict_talent(age, height_cm, weight_kg, sprint_30m_sec, vertical_jump_cm, agility_t_sec, coopers_m):
    if clf_model is None or reg_model is None:
        return "ابتدا CSV را آپلود کنید و «آموزش مدل‌ها» را بزنید."

    X = np.array([[age,height_cm,weight_kg,sprint_30m_sec,vertical_jump_cm,agility_t_sec,coopers_m]], dtype=float)
    p = float(clf_model.predict_proba(X)[0,1])
    scores = np.clip(reg_model.predict(X)[0], 0, 100)

    pose_text = ""
    if last_pose_feats is not None:
        pose_text = f"\n(آخرین Pose: hip_sway_mean={last_pose_feats.get('hip_sway_mean',0):.4f} | hip_conf={last_pose_feats.get('hip_conf',0):.2f})"

    return (
        f"حالت ۱ | احتمال قبولی آکادمی: {p*100:.1f}%\n"
        f"حالت ۳ | Speed: {scores[0]:.1f} / 100\n"
        f"حالت ۳ | Power: {scores[1]:.1f} / 100\n"
        f"حالت ۳ | Endurance: {scores[2]:.1f} / 100"
        + pose_text
    )

with gr.Blocks(title="استعداد‌یابی فوتبال (Colab)") as demo:
    gr.Markdown("""
# پروژه نهایی درس یادگیری ماشین | اپلیکیشن آنلاین استعداد‌یابی رشته فوتبال
 دانشگاه آزاد اسلامی – واحد علوم و تحقیقات | درس: یادگیری ماشین | استاد: دکتر مهدی اسلامی| نام و نام خانوادگی: علی اللهی قلی | شماره دانشجویی: 40112340119097
""")

    with gr.Tab("مرحله ۱: متن"):
        age = gr.Number(value=16, label="سن")
        height = gr.Number(value=175, label="قد (cm)")
        weight = gr.Number(value=65, label="وزن (kg)")
        foot = gr.Dropdown(["راست","چپ","هردو"], value="راست", label="پای غالب")
        sprint = gr.Number(value=4.6, label="اسپرینت ۳۰ متر (ثانیه)")
        jump = gr.Number(value=40, label="پرش عمودی (cm)")
        coop = gr.Number(value=2500, label="کوپر (متر)")
        agil = gr.Number(value=11.2, label="T-test (ثانیه)")

        btn1 = gr.Button("پیشنهاد پست")
        out_pos = gr.Textbox(label="پست پیشنهادی")
        out_exp = gr.Textbox(label="توضیحات", lines=6)

        btn1.click(
            recommend_position_text,
            inputs=[age,height,weight,foot,sprint,jump,coop,agil],
            outputs=[out_pos,out_exp]
        )

    with gr.Tab("مرحله ۲: Pose (عکس یا ویدئو)"):
        img = gr.Image(type="filepath", label="آپلود تصویر (اختیاری)")
        vid = gr.Video(label="آپلود ویدئو (اختیاری)")
        btn2 = gr.Button("تحلیل Pose")
        out2 = gr.Textbox(label="خروجی Pose", lines=4)
        btn2.click(ui_pose, inputs=[img, vid], outputs=[out2])

    with gr.Tab("مرحله ۳: استعداد (حالت ۱ و ۳)"):
        csv_file = gr.File(label="آپلود CSV دیتاست")
        btn_train = gr.Button("آموزش مدل‌ها")
        train_status = gr.Textbox(label="وضعیت آموزش")
        btn_train.click(ui_train, inputs=[csv_file], outputs=[train_status])

        btn_pred = gr.Button("پیش‌بینی استعداد")
        out3 = gr.Textbox(label="خروجی نهایی", lines=7)
        btn_pred.click(
            ui_predict_talent,
            inputs=[age,height,weight,sprint,jump,agil,coop],
            outputs=[out3]
        )

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://38580b421a936e2566.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
